In [ ]:
import cv2
import cvzone
from cvzone.HandTrackingModule import HandDetector
import pygame

# Initialize pygame
pygame.init()

# Set the window size and title
pygame.display.set_mode((800, 600))
pygame.display.set_caption("SET YOUR HAIR")

# Load the song
song_path = r"C:\Users\sharm\Music\old_SOTY-Ishq Wala Love.mp3"
pygame.mixer.music.load(song_path)

# Initialize the hand detector
detector = HandDetector(detectionCon=0.8)

# Shahrukh Khan's iconic pose landmarks (wrist, elbow, and shoulder)
pose_landmarks = [4, 5, 6]

# Main loop
cap = cv2.VideoCapture(0)
while True:
    # Read the frame from the camera
    success, frame = cap.read()
    if not success:
        break

    # Detect hands in the frame
    frame = cv2.flip(frame, 1)
    hands, _ = detector.findHands(frame)

    if hands:
        hand = hands[0]
        landmarks = hand["lmList"]
        fingers = detector.fingersUp(hand)

        # Check if the hand is in Shahrukh Khan's iconic pose
        if all(landmarks[i][2] > landmarks[i + 1][2] for i in pose_landmarks) and all(fingers):
            # Play the song
            pygame.mixer.music.play()

            # Display a message on the frame
            cvzone.cornerRect(frame, (20, 20, 180, 80), 20, rt=0)
            cv2.putText(frame, "Set your hair", (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow("Shahrukh Khan Iconic Pose Detection", frame)

    # Check for key press events
    if cv2.waitKey(1) == ord("q"):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Load the movie reviews dataset
data = pd.read_csv("movie_reviews.csv")  # Replace with the actual path to your dataset

# Preprocess the data
reviews = data["review"].values
labels = data["sentiment"].values

# Split the data into training and testing sets
train_reviews, test_reviews, train_labels, test_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Tokenize the reviews
tokenizer = Tokenizer(num_words=10000)  # Limit vocabulary size to the top 10,000 words
tokenizer.fit_on_texts(train_reviews)
train_sequences = tokenizer.texts_to_sequences(train_reviews)
test_sequences = tokenizer.texts_to_sequences(test_reviews)

# Pad the sequences
max_sequence_length = 100  # Limit the maximum sequence length to 100 words
train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=32, input_length=max_sequence_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=5, batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Save the trained model
model.save("sentiment_analysis_model.h5")
print("Model saved successfully.")
